#### Observe that we can get ETF's close of a day with the link format :
https://query1.finance.yahoo.com/v8/finance/chart/BND?symbol=BND&period1=1540963800&period2=1552973400&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=Kiilb9S0iNl&corsDomain=finance.yahoo.com ,where its period1&2 are in timestamp format.
#### The daily closing time is 13:30:00, timezone is EST.
#### We need to define the date range we want in timestamp format.

In [19]:
import datetime, time, pytz

EST = pytz.timezone('America/New_York')

start_date = "2015-12-31 13:30:00"
end_date = str(datetime.datetime.now().date()) + " 13:30:00"

start = time.strptime(start_date, "%Y-%m-%d %H:%M:%S")
end = time.strptime(end_date, "%Y-%m-%d %H:%M:%S")

start_timeStamp = str(int(time.mktime(start)))
end_timeStamp = str(int(time.mktime(end)))
#print(start_timeStamp)
#print(end_timeStamp)



#### Now write a function to parse the JSON text, return the data we want (dataFrames of different ETFs include date and closing price).


In [20]:
import json
import requests
import pandas as pd


def process_json(symbol_series, start, end):    

    all_data = list() # All DataFrames of different ETFs are stored in all_data.
    for symbol in symbol_series:
        date = list()
        #print('start crawling '+ symbol)        
        # get link
        url = 'https://query1.finance.yahoo.com/v8/finance/chart/'+ symbol + '?symbol='+ symbol +'&period1='+ start + '&period2='+ end +'&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=Kiilb9S0iNl&corsDomain=finance.yahoo.com'
        
        # request for document
        res = requests.get(url)  
        
        # 将 JSON 對象轉換為 Python 字典
        json_content = json.loads(res.text)        
    
        # get closing price
        adjclose = json_content['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']
        
        # get timestamp and convert to 'yyyy-mm-dd' format
        timestamp_list = json_content['chart']['result'][0]['timestamp']    
        for timestamp in timestamp_list:
            dt_tmp = datetime.datetime.fromtimestamp(timestamp, EST)
            date.append(dt_tmp.strftime("%Y-%m-%d"))
        
        # turn data into pandas dataframe
        data = {'Date' : date, symbol : adjclose}   
        df = pd.DataFrame.from_dict(data, orient='index')
        df = df.transpose()
        
        all_data.append(df)
    
    return all_data


#### Crawl over the ETFs on the site, store the data in variable "all_frames" .


In [21]:
# read file
file = pd.read_csv('Treasuries ETF List (51).csv')

# Screening for ETFs that existed before the end of 2015
mask = file['Inception'] < "2015-12-31"

# symbol of ETFs that existed before the end of 2015
Symbols = file[mask]['Symbol']

# crawl over the ETFs on the site
all_frames = process_json(Symbols, start_timeStamp, end_timeStamp)

# merge all dataframes and show the result
for i in range(1,len(all_frames)):
    all_frames[0] = pd.merge(all_frames[0],all_frames[i],on='Date',how='outer')
all_frames[0]
    

Date      SHV      SHY      IEF      TLT      BIL     GOVT  \
0    2015-12-31  106.858    81.23  99.2156  111.317  88.8459  23.8303   
1    2016-01-04  106.858   81.336  99.6385   112.12  88.8653  23.8874   
2    2016-01-05  106.867  81.2878  99.6103  111.668  88.8653  23.8588   
3    2016-01-06  106.848   81.336  100.212  113.172  88.8264  23.9539   
4    2016-01-07  106.858  81.3648  100.428  113.375  88.8459  24.0015   
5    2016-01-08  106.867  81.4226  100.681  113.883  88.8459    24.03   
6    2016-01-11  106.858  81.4515  100.362  112.637  88.8459   23.992   
7    2016-01-12  106.867  81.4612  100.813  114.262  88.8653   24.068   
8    2016-01-13  106.867  81.5093  101.208  115.388  88.8459  24.1346   
9    2016-01-14  106.887  81.5285  101.001  114.308  88.8264  24.0966   
10   2016-01-15  106.896  81.5959  101.471   116.09  88.8264  24.1726   
11   2016-01-19  106.935  81.5863  101.339   115.73  88.8459  24.1631   
12   2016-01-20  106.945  81.6441  101.837  116.948  88.8459  24.2487   
13   2016-01-21  106.945  81.6345  101.583  116.154  88.8653  24.2107   
14   2016-01-22  106.925  81.6056  101.358  115.711  88.8264  24.1822   
15   2016-01-25  106.925  81.6056  101.696  116.339  88.8459  24.2107   
16   2016-01-26  106.925  81.6441  101.771  116.394  88.8264  24.2297   
17   2016-01-27  106.925  81.6633  101.847  116.367  88.8459  24.2202   
18   2016-01-28  106.925  81.6633  101.978  116.533  88.8459  24.2677   
19   2016-01-29  106.935  81.7597  102.523  117.521  88.8653  24.3438   
20   2016-02-01  106.915  81.7163  102.287  117.161  88.8264  24.2867   
21   2016-02-02  106.944  81.8223  103.012  119.334  88.8264   24.439   
22   2016-02-03  106.954  81.8415  102.899  118.345  88.8459  24.4009   
23   2016-02-04  106.954  81.8608  103.143  118.918  88.8653  24.4485   
24   2016-02-05  106.934  81.8415  103.275  119.057  88.8653  24.4581   
25   2016-02-08  106.934  81.9379  104.047  121.619  88.8459  24.5818   
26   2016-02-09  106.954   81.909  104.065  121.748  88.8653  24.5818   
27   2016-02-10  106.944   81.909  104.376  122.821  88.8459  24.6485   
28   2016-02-11  106.954   81.986  104.856  123.681  88.8653  24.7151   
29   2016-02-12  106.963  81.8897  104.056  121.628  88.8459  24.5818   
..          ...      ...      ...      ...      ...      ...      ...   
783  2019-02-12  110.195  83.5161  104.111  121.304  91.3402   24.778   
784  2019-02-13  110.205  83.4563  103.851  120.845  91.3402   24.758   
785  2019-02-14  110.235  83.5661   104.34  121.533  91.3601  24.8179   
786  2019-02-15  110.255  83.5261   104.24  121.723  91.3601  24.8179   
787  2019-02-19  110.265   83.586   104.42  122.032  91.3801  24.8478   
788  2019-02-20  110.265   83.576    104.4  121.683  91.3801  24.8378   
789  2019-02-21  110.295  83.5461  104.091  120.595  91.3901   24.778   
790  2019-02-22  110.295   83.606    104.4  121.314  91.4101  24.8279   
791  2019-02-25  110.295   83.596   104.24  120.954  91.4001  24.8079   
792  2019-02-26  110.295  83.6359   104.55  121.553  91.4101  24.8478   
793  2019-02-27  110.325   83.616   104.18  120.176    91.42  24.7879   
794  2019-02-28  110.325   83.596  103.911  119.767    91.44   24.748   
795  2019-03-01   110.34    83.54   103.57   118.66    91.45    24.68   
796  2019-03-04   110.36    83.56   103.89   119.59    91.44    24.74   
797  2019-03-05   110.35    83.57   103.95   119.84    91.45    24.75   
798  2019-03-06   110.37    83.63   104.24   120.28    91.46    24.79   
799  2019-03-07   110.39     83.7   104.59   121.06    91.48    24.85   
800  2019-03-08   110.41    83.73   104.74   121.57    91.49    24.89   
801  2019-03-11   110.39    83.71   104.66   121.21    91.49    24.84   
802  2019-03-12   110.41    83.74   104.95   122.06     91.5    24.92   
803  2019-03-13   110.43    83.75   104.91   121.83    91.51    24.91   
804  2019-03-14   110.43    83.77   104.76   120.95    91.51    24.87   
805  2019-03-15   110.46     83.8   105.09   121.75    91.52    24.92

#### Save as pkl file.

In [22]:
import pickle

with open('etf_close.pkl', 'wb') as f:
    pickle.dump(all_frames[0], f)